1. Process Wiktionary dump to extract synonym relations for a random language (not English, Ukrainian or Russian :) - requires application of XML SAX parsing

In [1]:
!wget -O wiktionary.xml.bz2 https://dumps.wikimedia.org/bgwiktionary/20180301/bgwiktionary-20180301-pages-articles-multistream.xml.bz2 

--2018-03-19 10:42:11--  https://dumps.wikimedia.org/bgwiktionary/20180301/bgwiktionary-20180301-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.11
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42317925 (40M) [application/octet-stream]
Saving to: ‘wiktionary.xml.bz2’

wiktionary.xml.bz2  100%[===================>]  40.36M  2.33MB/s    in 32s     

2018-03-19 10:42:44 (1.25 MB/s) - ‘wiktionary.xml.bz2’ saved [42317925/42317925]



In [2]:
from lxml import etree
import bz2
import re

In [3]:
# Exploration - printing first 100 lines
# i = 0;
# input_file = bz2.open("wiktionary.xml.bz2")
# while (i < 100):
#     i += 1
#     print(input_file.readline().decode('utf-8'))
# Exploration - printing 200 lines after <ns> element
# i = 0;
# last_print = 0
# printF = False
# file = bz2.open("wiktionary.xml.bz2")
# while (i < 3000):
#     line = file.readline().decode('utf-8')
#     if line.strip() in ['<ns>0</ns>']: #,'<ns>-1</ns>', , '<ns>4</ns>', '<ns>14</ns>']:
#         printF = True
#         last_print = i
#     if printF:
#         i += 1
#         print(line)
#         if (i - last_print) > 200:
#             printF = False

In [34]:
header_re = re.compile(r"===\s*Синоними\s*===", re.M | re.UNICODE)
extra_re = re.compile(r"'''Синоними:?'''", re.M | re.UNICODE)
bullets_re = re.compile(r"^\s*\*", re.M | re.UNICODE)
content_re = re.compile(r"\[\[?([\w\-\s\(\),':]*)\]?\]?", re.M | re.UNICODE)
extra_keys_re = re.compile(r"[А-Я ]+\s*=[\s\n]*", re.M | re.UNICODE)
def cleanup_syns_list(raw_string):
    raw_string = re.sub(header_re, "", raw_string)
    raw_string = re.sub(extra_re, "", raw_string)
    raw_string = re.sub(bullets_re, ", ", raw_string)
    raw_string = re.sub(content_re, r'\1', raw_string)
    raw_string = re.sub(extra_keys_re, "", raw_string)
    return re.sub(r"\s+,\s+", ", ", " ".join(raw_string.split("\n")).strip(", "))

word_re = re.compile(r"ID\s+=\s+(\w+)", re.M | re.UNICODE)
syns_re = re.compile(r"\| (?:СИНОНИМИ|СРОДНИ\s+ДУМИ|ДРУГИ) = ([^\|\}]+)", re.M | re.UNICODE | re.I)
syns_list_re = re.compile(r"===\s*Синоними\s*===\n(\s*\*[^\n]*\n)*", re.M | re.UNICODE | re.I)
def extract_word_and_syns(raw_text):
    result = word_re.search(text)
    word = result and result.group(1)
    syns = None
    if word:
        if "СИНОНИМИ" in text or "СРОДНИ ДУМИ" in text or "ДРУГИ" in text:
            result = syns_re.search(text)
            if (result and result.group(1).strip()):
                syns = cleanup_syns_list(result.group(1).strip())
        if "Синоними" in text:
            result = syns_list_re.search(text)
            if result and result.group(0).strip():
                syns = cleanup_syns_list(result.group(0).strip())
    return (word, syns)

In [36]:
waiting_ns = False
waiting_text = False
text = ''
title = None
with open('bg_wiktionary_syns.txt', 'w') as output_file:
    for (event, elem) in etree.iterparse(bz2.open("wiktionary.xml.bz2"), events=['start', 'end']):
        if (event == 'start' and elem.tag[-4:] == 'page'):
            waiting_ns = True
            continue
        if (event == 'end' and elem.tag[-4:] == 'page'):
            waiting_ns = False
            waiting_text = False
            title = None
            continue
        if (event == 'end' and waiting_ns and elem.tag[-5:] == 'title'):
            title = elem.text
            continue
        if (waiting_ns and event == 'end' and 'ns' == elem.tag[-2:]):
            if elem.text.strip() == '0':
                waiting_text = True
            continue
        if (waiting_text and event == 'end' and 'text' == elem.tag[-4:]):
            text = elem.text
            if not text:
                continue
            (word, syns) = extract_word_and_syns(text)
            if not syns:
                continue
            output_file.write("{} (https://bg.wiktionary.org/wiki/{}): {}\n".format(word, title, syns))
            continue

In [39]:
text = """
{{-bg-}}
{{Съществително нарицателно име|
  ID = боб
| ЗНАЧЕНИЕ =
# Растение и плода на това растение.
| РОД = м
| ТИП = 1
| ЕТИМОЛОГИЯ = 
| ИЗРАЗИ = 
Купих  боб , ще направя бобена чорба. 
Засях боб в градината. 
| ПРЕВОД1 =
* английски: [[bean]] (Едно зърно.) , [[:en:bean|(en)]][[beans]] [[:en:beans|(en)]]
* арабски: [[ ‏فَاصُولِيَا (faʂúːliya) , ‏لُوبِيَا (láːbiya)]] [[:ar:فَاصُولِيَا|(ar)]]
* арменски: [[]] 
* африкаанс: [[]] [[:af:|(af)]]
* белоруски: [[]] [[:be:|(be)]]
* гръцки: [[φασόλι (фасоли)]] [[:el:φασόλι|(el)]]
* датски: [[ bønne ]](ед) [[:da:bønne|(da)]],[[bønner pl]](мн.)  [[:da:bønner|(da)]]
* есперанто: [[fabo]] [[:eo:fabo|(eo)]]
* естонски: [[]] [[:et:|(et)]]
* иврит: [[שעועית (še'u'ít)]] 
* индонезийски: [[]]
* ирландски: [[]] 
* исландски: [[]] 
* испански: [[haba]] [[:es:haba|(es)]] ,[[frijol]] [[:es:frijol|(es)]]
* италиански: [[fagiolo]] [[:it:fagiolo|(it)]]
* китайски: [[]] [[:zh:|(zh)]]
* корейски: [[콩 (kong)]] [[:ko:|(ko)]]
* латвийски: [[]] [[:lv:|(lv)]]
* латински: [[]] [[:la:|(la)]]
* литовски: [[]] [[:lt:|(lt)]]
| ПРЕВОД2 =
* немски:  [[Bohne]] [[:de:Bohne|(de)]], [[Bohnen]] (мн.)[[:de:Bohnen|(de)]]
* норвежки: [[bønne]] [[:no:bønne|(no)]]
* персийски: [[لوبیا (lubiyā), باقالا (bāγālā)]] [[:fa:لوبیا |(fa)]]
* полски: [[fasola]] [[:pl:fasola|(pl)]]
* португалски: [[feijão]] [[:pt:feijão|(pt)]]
* румънски: [[fasolea]] [[:ro:fasolea|(ro)]], [[bob]] [[:ro:bob| (ro)]]
* руски:  [[боб]] [[:ru:боб|(ru)]] ,[[бобы ]](мн.) [[:ru:бобы |(ru)]] ,[[фасоль]][[:ru:фасоль|(ru)]] ,[[фасоли]](мн.) [[:ru:фасоли|(ru)]]
* словашки: [[fazul’a]] [[:sk:fazul’a|(sk)]]
* словенски:[[]] [[:sl:|(sl)]]
* сръбски: [[пасуљ]] [[:sr:пасуљ|(sr)]]
* тайландски: [[ถั่ว (thuàà)]] [[:th:ถั่ว|(th)]]
* турски: [[fasulye]] [[:tr:fasulye|(tr)]]
* унгарски: [[bab, paszuly]] [[:hu:bab|(hu)]]
* фински: [[papu]] [[:fi:papu|(fi)]]
* френски: [[haricot]] [[:fr:haricot|(fr)]]
* холандски: [[boon]] [[:nl:boon|(nl)]]
* хърватски: [[pasulj]] [[:hr:pasulj|(hr)]]
* чешки: [[fazole]] [[:cs:fazole|(cs)]]
* шведски: [[böna]] [[:sv:böna|(sv)]] , [[bönor]](мн.) [[:sv:bönor|(sv)]] 
* японски: [[豆 (まめ, mamé)]] [[:ja:|(ja)]]
| СИНОНИМИ = [[фасул]]
| СРОДНИ ДУМИ = 
| ПРОИЗВОДНИ ДУМИ = 
| ДРУГИ = 
}}

"""
print(extract_word_and_syns(text))

('боб', 'фасул')


In [38]:
text = """
{{-bg-}}
{{Съществително нарицателно име|
  ID = чифут
| ЗНАЧЕНИЕ =
# Синоним на "[[евреин]]". От албански.
| РОД = 
| ТИП = 
| СРИЧКИ = 
| ЕТИМОЛОГИЯ = от албански 
| ИЗРАЗИ = 
| ПРЕВОД1 = 
* руски: 
* албански: | ПРЕВОД2 =
| СИНОНИМИ = '''Синоними'''| СРОДНИ ДУМИ = ПРОИЗВОДНИ ДУМИ = 
| ДРУГИ = се среща на [[w:Балкани|Балканите]] [[w:турци|турци]] &mdash; в разказа на [[w:хървати|хърватския]] писател от [[w:Бачка|Бачка]] на  [[w:Павле Башич-Палкович|Павле Башич-Палкович]].

В легендите "[http://liternet.bg/folklor/sbornici/senki/128.htm Цар Константин намира кръста Христов]" и "[http://liternet.bg/folklor/sbornici/senki/115.htm Възкресение Христово]" са използвани производни на думата "чифут".
}}

"""
extract_word_and_syns(text)

('чифут', '')